<a href="https://colab.research.google.com/github/paulgureghian/Ludwig_Models/blob/master/Medetec_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# List .txt files in the root.
#
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "'15hEQjiKwk3oXRd-6hgv6G7qWLmT7z1HG' in parents"}).GetList()
for file in listed:
  print('title {}, id {}'.format(file['title'], file['id']))

title requirements.txt, id 1kBh1OAS1H_XJYSqkfPwZ91j9HHvotzc3
title Dockerfile, id 1pDDrPUasSSM2XrAl6UfOfoupQkSFA_r6
title LICENSE, id 1IRTQWTLpq6iITv-RjBUST8I8_QuFnKSZ
title README.md, id 167UDmGw6oFPdBaclH-gPa33oqMKSAVE8
title setup.py, id 1dBrEdSt5miSwdOsy1mAHx7NPNFSGf9Tw
title .gitignore, id 1J5LPoUR1Hb321TnYEQ1sOsZ19cnN8CJS
title tests, id 1RZSo1Y7iHOkbjxB2lPRu-qVR8j1Gfdno
title mkdocs, id 1x7757FAXDdjkFpvD9PHFAhf83xxe13Jg
title .git, id 1U-M2hI3gqLdpyV6sayIIqlOrKaIqr1qt
title ludwig, id 13OUsd8j59N1QUtUeP9MVjoFpsq5OEpeG
title docs, id 1mnDOUZfykSswQc8TdtG1BHf6Fsq6GRhT


In [0]:
requirements = drive.CreateFile({'id' : '1kBh1OAS1H_XJYSqkfPwZ91j9HHvotzc3'}) 
requirements.GetContentFile('requirements.txt') 

print('File successfully imported.') 

In [0]:
!pip install -r requirements.txt

In [0]:
!python -m spacy download en

In [0]:
setup = drive.CreateFile({'id' : '1dBrEdSt5miSwdOsy1mAHx7NPNFSGf9Tw'})
setup.GetContentFile('setup.py') 

In [0]:
!python setup.py install